In [1]:
import requests
import mysql.connector
from mysql.connector import Error

In [2]:
HOST = 'localhost'
DATABASE = 'code_review'
USER = 'root'
PASSWORD = 'comp90055_MSR'

NAME = ['kernel', 'ozlabs', 'ffmpeg']
CATEGORY = ['projects', 'series', 'patches']
INVALID_PAGE = {
    "detail": "Invalid page."
}

In [3]:
def connect(host, database, user, password):
    """ Connect to MySQL database """
    connection = None
    try:
        connection = mysql.connector.connect(host=HOST,
                                       database=DATABASE,
                                       user=USER,
                                       password=PASSWORD)
        if connection.is_connected():
            print('Connected to MySQL database')

    except Error as e:
        print(e)

    # finally:
    #     if conn is not None and conn.is_connected():
    #         conn.close()
    return connection

In [4]:
def deactivate_quote(text):
    if text:
        split_text = list(text)
        for i in range(len(split_text)):
            if (split_text[i] in ["'", '"'] and split_text[i - 1] != '\\') or (split_text[i - 3:i + 1] == '\\\\\"'):
                split_text[i] = '\\' + split_text[i]
        temp_text = ''.join(split_text)
        return temp_text
    else:
        return ''

In [5]:
def crawl_project_data(json_project, connection, cursor):
    print(f"retrieving project: {json_project['url']}")
    # project info
    ## delete the last slash in the api url
    project_original_id = json_project['url'][:-1]
    project_name = deactivate_quote(json_project['name'])
    project_repo_url = deactivate_quote(json_project['webscm_url'])
    project_list_id = deactivate_quote(json_project['list_id'])
    project_list_address = deactivate_quote(json_project['list_email'])
    project_web_url = deactivate_quote(json_project['web_url'])

    query_insert_project = f'''INSERT IGNORE INTO project VALUES(null, "{project_original_id}", "{project_name}", "{project_repo_url}", "{project_web_url}", "{project_list_id}", "{project_list_address}")'''
    cursor.execute(query_insert_project)

    # maintainer_info
    maintainers = json_project['maintainers']
    for maintainer in maintainers:
        maintainer_original_id = maintainer['url'][:-1]
        maintainer_username = deactivate_quote(maintainer['username'])
        maintainer_email = deactivate_quote(maintainer['email'])

        # maintainer_first_name = maintainer['first_name']
        # maintainer_last_name = maintainer['last_name']
        # if maintainer_first_name == '' and maintainer_last_name == '':
        #     maintainer_last_name = maintainer_username
        # maintainer_full_name = f"{maintainer_first_name} {maintainer_last_name}".strip()

        # # user table
        # query_user = f"INSERT IGNORE INTO user(original_id, name, first_name, last_name)" \
        #              f"VALUES ({maintainer_original_id}, '{maintainer_full_name}', '{maintainer_first_name}', '{maintainer_last_name}')"
        # cursor.execute(query_user)

        # # get user id
        # query_fetch_user_id = f"SELECT id FROM user WHERE first_name = '{maintainer_first_name}' AND last_name = '{maintainer_last_name}'"
        # cursor.execute(query_fetch_user_id)
        # user_id = cursor.fetchall()[0][0]

        # account table
        query_insert_account = f'''INSERT IGNORE INTO account VALUES (null, "{maintainer_original_id}", "{maintainer_email}", "{maintainer_username}")'''
        cursor.execute(query_insert_account)

        # get account id
        query_fetch_project_id = f'''SELECT id FROM account WHERE original_id = "{maintainer_original_id}"'''
        cursor.execute(query_fetch_project_id)
        maintainer_account_id = cursor.fetchall()[0][0]

        # get project id
        query_fetch_project_id = f'''SELECT id FROM project WHERE original_id = "{project_original_id}"'''
        cursor.execute(query_fetch_project_id)
        project_id = cursor.fetchall()[0][0]

        query_maintainer = f'''INSERT IGNORE INTO maintainer VALUES ({maintainer_account_id}, {project_id})'''
        cursor.execute(query_maintainer)

    connection.commit()


def crawl_series_data(json_series, connection, cursor):
    print(f"retrieving series: {json_series['url']}")
    # print(json_series)
    # series info
    series_original_id = json_series['url'][:-1]
    series_name = deactivate_quote(json_series['name'])
    series_created_date = json_series['date']
    series_version = json_series['version']
    series_total = json_series['total']
    series_received_total = json_series['received_total']

    #get cover letter content
    if json_series['cover_letter']:
        cover_letter_url = json_series['cover_letter']['url']
        cover_detail = requests.get(cover_letter_url).json()
        series_cover_letter_content = deactivate_quote(cover_detail['content'])
    else: series_cover_letter_content = None

    # get project id
    series_proj_original_id = json_series['project']['url'][:-1]
    # series_proj_name = json_series['project']['name']
    query_fetch_project_id = f'''SELECT id FROM project WHERE original_id = "{series_proj_original_id}"'''
    cursor.execute(query_fetch_project_id)
    series_project_id = cursor.fetchall()[0][0]

    # submitter info
    series_submitter_original_id = json_series['submitter']['url'][:-1]
    series_submitter_email = deactivate_quote(json_series['submitter']['email'])
    series_submitter_name = deactivate_quote(json_series['submitter']['name'])

    # insert account if not exist
    query_insert_account = f'''INSERT IGNORE INTO account VALUES (null, "{series_submitter_original_id}", "{series_submitter_email}", "{series_submitter_name}")'''
    cursor.execute(query_insert_account)

    # fetch account_id from account table
    query_fetch_account_id = f'''SELECT id FROM account WHERE original_id = "{series_submitter_original_id}"'''
    cursor.execute(query_fetch_account_id)
    series_submitter_account_id = cursor.fetchall()[0][0]

    query_insert_series = f'''INSERT IGNORE INTO series VALUES (null, "{series_original_id}", "{series_name}", "{series_created_date}", {series_version}, {series_total}, {series_received_total}, "{series_cover_letter_content}", {series_project_id}, {series_submitter_account_id})'''
    cursor.execute(query_insert_series)

    connection.commit()


def crawl_comment_data(json_comment, patch_original_id, connection, cursor):
    print(f"retrieving comment, id: {json_comment['web_url']}")
    #comment info
    comment_original_id = json_comment['web_url']
    comment_msg_id = json_comment['msgid']
    comment_msg_content = deactivate_quote(json_comment['content'])
    comment_date = json_comment['date']
    comment_subject = deactivate_quote(json_comment['subject'])
    comment_reply_to_msg_id = ''
    if 'In-Reply-To' in json_comment['headers'].keys():
        in_reply_to = json_comment['headers']['In-Reply-To']
        if in_reply_to[:2] == '\n ':
            comment_reply_to_msg_id = in_reply_to[2:]

    # TODO get mailing list id

    # get patch id
    query_fetch_patch_id = f"SELECT id FROM patch WHERE original_id = '{patch_original_id}'"
    cursor.execute(query_fetch_patch_id)
    # print(cursor.fetchall())
    comment_patch_id = cursor.fetchall()[0][0]

    # TODO get change id

    # get submitter account id
    # insert account if not exist
    comment_submitter_original_id = json_comment['submitter']['url'][:-1]
    comment_submitter_username = deactivate_quote(json_comment['submitter']['name'])
    comment_submitter_email = deactivate_quote(json_comment['submitter']['email'])

    query_insert_account = f'''INSERT IGNORE INTO account VALUES (null, "{comment_submitter_original_id}", "{comment_submitter_email}", "{comment_submitter_username}")'''
    cursor.execute(query_insert_account)

    # fetch account_id from account table
    query_fetch_account_id = f'''SELECT id FROM account WHERE original_id = "{comment_submitter_original_id}"'''
    cursor.execute(query_fetch_account_id)
    comment_submitter_account_id = cursor.fetchall()[0][0]

    query_insert_comment = f'''INSERT IGNORE INTO comment VALUES (null, "{comment_original_id}", "{comment_msg_id}", "{comment_msg_content}", "{comment_date}", "{comment_subject}", "{comment_reply_to_msg_id}", null, "{comment_patch_id}", null, "{comment_submitter_account_id}")'''

    try:
        cursor.execute(query_insert_comment)
        # connection.commit()
    except Error as e:
        print(query_insert_comment)
        print(e)



def crawl_patch_data(json_patch, connection, cursor):
    print(f"retrieving patch {json_patch['url']}")
    #patch info
    patch_original_id = json_patch['url'][:-1]
    patch_name = deactivate_quote(json_patch['name'])
    patch_state = deactivate_quote(json_patch['state'])
    patch_date = json_patch['date']
    patch_msg_id = json_patch['msgid']
    patch_msg_content = deactivate_quote(json_patch['content'])
    patch_code_diff = deactivate_quote(json_patch['diff'])

    # TODO get change id
    # TODO get mailing list id

    # get series id
    if json_patch['series']:
        patch_series_original_id = json_patch['series']['url'][:-1]
        query_fetch_series_id = f'''SELECT id FROM series WHERE original_id = "{patch_series_original_id}"'''
        cursor.execute(query_fetch_series_id)
        if cursor.fetchall():
            patch_series_id = cursor.fetchall()[0][0]
    else:
        patch_series_id = 'null'

    # get submitter account id
    # insert account if not exist
    patch_submitter_original_id = json_patch['submitter']['url'][:-1]
    patch_submitter_username = deactivate_quote(json_patch['submitter']['name'])
    patch_submitter_email = deactivate_quote(json_patch['submitter']['email'])

    query_insert_account = f'''INSERT IGNORE INTO account VALUES (null, "{patch_submitter_original_id}", "{patch_submitter_email}", "{patch_submitter_username}")'''
    cursor.execute(query_insert_account)

    # fetch account_id from account table
    query_fetch_account_id = f'''SELECT id FROM account WHERE original_id = "{patch_submitter_original_id}"'''
    cursor.execute(query_fetch_account_id)
    patch_submitter_account_id = cursor.fetchall()[0][0]

    query_insert_patch = f'''INSERT IGNORE INTO patch VALUES (null, "{patch_original_id}", "{patch_name}", "{patch_state}", "{patch_date}", "{patch_msg_id}", "{patch_msg_content}", "{patch_code_diff}", null, null, {patch_series_id}, {patch_submitter_account_id})'''

    cursor.execute(query_insert_patch)

    comment_url = json_patch['comments']
    comment_list = requests.get(comment_url).json()
    if comment_list:
        for comment in comment_list:
            crawl_comment_data(comment, patch_original_id, connection, cursor)

    connection.commit()

def retrieved_item(name, cursor):
    query = f'''SELECT original_id from {name}'''
    cursor.execute(query)
    result = [item[0] for item in cursor.fetchall()]
    return result

def crawl_data(name, category, connection, cursor):
    
    platform_url = f'https://patchwork.{name}.org'
    api_url_base = f'{platform_url}/api/{category}/?page='
    page_num = 1
    flag = True
    
    retrieved_project = retrieved_item('project', cursor)
    retrieved_series = retrieved_item('series', cursor)
    retrieved_patch = retrieved_item('patch', cursor)

    while flag and page_num <= 5:
        url = api_url_base + f'{page_num}'
        response = requests.get(url).json()
        if type(response) == dict and response == INVALID_PAGE:
            flag = False
        else:
            for entity in response:
                entity_api_url = entity['url']
                entity_detail = requests.get(entity_api_url).json()
                if category == 'projects':
                    if not (entity_detail['url'][:-1] in retrieved_project):
                        crawl_project_data(entity_detail, connection, cursor)
                elif category == 'series':
                    if not (entity_detail['url'][:-1] in retrieved_series):
                        crawl_series_data(entity_detail, connection, cursor)
                elif category == 'patches':
                    if not (entity_detail['url'][:-1] in retrieved_patch):
                        crawl_patch_data(entity_detail, connection, cursor)

            page_num += 1

    connection.commit()

In [7]:
conn = connect(HOST, DATABASE, USER, PASSWORD)
cursor = conn.cursor()

for cate in CATEGORY:
    crawl_data(NAME[2], cate, conn, cursor)

cursor.close()
conn.close()

Connected to MySQL database
retrieving project: FFmpeg
retrieving series: Matroska muxer patches
retrieving series: [FFmpeg-devel,V1,01/12] lavc/bsf: fix memory leak after av_dict_parse_string fail
retrieving series: [FFmpeg-devel,V1] lavfi/buffersrc: Remove redundancy free after ff_filter_frame fail
retrieving series: [FFmpeg-devel,V1] libavcodec/aacdec: fix HE-AAC decoder dirty date (for sbr) when flush
retrieving series: [FFmpeg-devel] configure: bump year
retrieving series: None
retrieving series: [FFmpeg-devel] avformat/image2: Upon request, make available extra metadata, fields related to input path to be used by filters.
retrieving series: [FFmpeg-devel,1/2] avdevice/xcbgrab: move some initialization code from pixfmt_from_pixmap_format to create_stream
retrieving series: [FFmpeg-devel,1/2] avcodec/smacker: Check space before decoding type
retrieving series: [FFmpeg-devel,v5,1/4] add AV_FRAME_DATA_USER_DATA_UNREGISTERED side data type
retrieving series: [FFmpeg-devel] Removes lin